In [ ]:
import cv2

img = cv2.imread("input.jpg")


In [ ]:
import pytesseract
from PIL import Image

# If tesseract is not in PATH, set it manually
# pytesseract.pytesseract.tesseract_cmd = r"/usr/bin/tesseract"

image_path = "/home/kawsar/Desktop/Image-Masking/POC-table-masking/src/images/ss-1.jpeg"
image_path = "/home/kawsar/Downloads/selected_sap_images/9325_screenshot.png"

img = Image.open(image_path)

text = pytesseract.image_to_string(img, lang="eng", config="--psm 12")
# remove empty lines
text = "\n".join([line for line in text.split("\n") if line.strip() != ""])

print(text)


In [ ]:
import cv2
import pytesseract

# image_path = "image.png"

img = cv2.imread(image_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Optional preprocessing
gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

text = pytesseract.image_to_string(gray, lang="eng", config="--psm 12")
text = "\n".join([line for line in text.split("\n") if line.strip() != ""])

print(text)


In [4]:
import cv2
import pytesseract
from pytesseract import Output

image_path = "/home/kawsar/Desktop/Image-Masking/POC-table-masking/outputs/cropped_images/cropped_5.png"
img = cv2.imread(image_path)

data = pytesseract.image_to_data(
    img,
    lang="eng",
    config="--oem 3 --psm 12",
    output_type=pytesseract.Output.DICT
)

lines = {}

# -----------------------------
# Group words into lines
# -----------------------------
for i in range(len(data["text"])):
    text = data["text"][i].strip()
    conf = int(data["conf"][i])

    # level 5 = word
    if not text or conf <= 0 or data["level"][i] != 5:
        continue

    key = (
        data["block_num"][i],
        data["par_num"][i],
        data["line_num"][i]
    )

    lines.setdefault(key, []).append(i)

line_texts = []

# -----------------------------
# Build line-level text + box
# -----------------------------
for indices in lines.values():
    texts = [data["text"][i] for i in indices]

    x_min = min(data["left"][i] for i in indices)
    y_min = min(data["top"][i] for i in indices)
    x_max = max(data["left"][i] + data["width"][i] for i in indices)
    y_max = max(data["top"][i] + data["height"][i] for i in indices)

    line_texts.append({
        "text": " ".join(texts),
        "box": [
            [x_min, y_min],
            [x_max, y_min],
            [x_max, y_max],
            [x_min, y_max],
        ],
        "prob": min(int(data["conf"][i]) for i in indices) / 100.0
    })

# -----------------------------
# Sort lines top → bottom
# -----------------------------
line_texts.sort(key=lambda x: x["box"][0][1])
for line in line_texts:
    print(line["text"], line["box"], line["prob"])

Cash Basis [[8, 15], [77, 15], [77, 25], [8, 25]] 0.54
Status, [[8, 30], [47, 30], [47, 40], [8, 40]] 0.38


Work With Order Headers [[8, 7], [180, 7], [180, 18], [8, 18]] 0.85
Layout [[886, 9], [918, 9], [918, 21], [886, 21]] 0.95
+ [[1024, 9], [1029, 9], [1029, 14], [1024, 14]] 0.33
Query: {All Records: [[1051, 9], [1160, 9], [1160, 21], [1051, 21]] 0.28
(No Layout) [[939, 10], [999, 10], [999, 21], [939, 21]] 0.9
= [[158, 33], [168, 33], [168, 34], [158, 34]] 0.67
YvQtoxXx [[4, 33], [139, 33], [139, 47], [4, 47]] 0.06
Row (2) Tools [[177, 33], [259, 33], [259, 47], [177, 47]] 0.56
a0 [[533, 65], [555, 65], [555, 79], [533, 79]] 0.19
BranchiPlant [[406, 67], [469, 67], [469, 76], [406, 76]] 0.77
(Order Number [[21, 69], [92, 69], [92, 78], [21, 78]] 0.64
oP [[188, 70], [203, 70], [203, 79], [188, 79]] 0.7
% [[118, 71], [121, 71], [121, 74], [118, 74]] 0.25
7 [[240, 71], [243, 71], [243, 74], [240, 74]] 0.55
‘Supplier [[21, 93], [61, 93], [61, 104], [21, 104]] 0.73
Records 1-10 > > [[9, 127], [131, 127], [131, 155], [9, 155]] 0.32
or [[142, 180], [154, 180], [154, 189], [142, 189]] 0.74
Orde

In [24]:
# write them into json file
import json
with open("output.json", "w") as f:
    json.dump(line_texts, f, indent=4)